# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
import cartopy
import geoviews
import pyproj

os.chdir('/Users/zan/Desktop/Data Analytics Course/python-api-challenge/WeatherPy')
print(os.getcwd()) 

# Import API key
from api_keys import geoapify_key



/Users/zan/Desktop/Data Analytics Course/python-api-challenge/WeatherPy


In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/zan/Desktop/Data Analytics Course/python-api-challenge/WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lng",                
    y="Lat",                
    geo=True,               
    tiles="OSM",           
    size=100,               
    color="City",      
    alpha=0.8,            
    hover_cols=["City", "Max Temp"], 
    title="City Weather Map"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
# Narrowed down cities based on the following criteria:
# - Temperature is between 30°C and 40°C
# - Wind speed is at least 5 mph

filtered_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 30) &  # Temperature >= 30°C
    (city_data_df["Max Temp"] <= 40) &  # Temperature <= 40°C
    (city_data_df["Wind Speed"] >= 5)   # Wind speed >= 5 mph
]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038
55,55,cayenne,4.9333,-52.3333,32.02,58,75,5.14,GF,1666108014
98,98,itarema,-2.9248,-39.9167,31.07,54,14,9.48,BR,1666108293
300,300,aquiraz,-3.9014,-38.3911,31.92,56,22,7.50,BR,1666108415
307,307,la romana,18.4273,-68.9728,30.20,79,75,5.14,DO,1666108418


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()



,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000  # Search radius in meters
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "apiKey": geoapify_key,               # Your Geoapify API key
    "limit": 1                            # Limit results to the nearest hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        # Extract the hotel name from the API response
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
lanzhou - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
colorado - nearest hotel: No hotel found
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
dobryanka - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: No hotel found
tasiilaq - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
nuevo laredo - nearest hotel: No hotel found
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: No hot

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
import hvplot.pandas
import pandas as pd

# Load the city data
city_data_df = pd.read_csv("/Users/zan/Desktop/Data Analytics Course/python-api-challenge/WeatherPy/output_data/cities.csv")

# Create a DataFrame with relevant data (assuming hotel_df is already prepared)
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df["Hotel Name"] = "Sample Hotel"  # Use actual hotel data if available

# Create an interactive map plot
map_plot = hotel_df.hvplot.points(
    x="Lng",                # Longitude column
    y="Lat",                # Latitude column
    geo=True,               # Enable Geo-plotting
    tiles="OSM",            # Use OpenStreetMap as the tile layer
    size=100,               # Marker size
    color="City",           # Color by city name
    alpha=0.8,              # Transparency
    hover_cols=["City", "Country", "Hotel Name", "Humidity"],  # Columns to display on hover
    title="City Map with Hotels"
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)